In [2]:
import os
%pwd

'/home/yousef/Kidney-Diseases-Classification/research'

In [3]:
os.chdir("../")

In [ ]:
from dataclasses import datclass
from pathlib import path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:str
    unzip_dir:str

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

        def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion
            create_directories([config.root_dir])
            data_ingestion_config = DataIngestionConfig(
                root_dir=Path(config.root_dir),
                source_URL=config.source_URL,
                local_data_file=Path(config.local_data_file),
                unzip_dir=Path(config.unzip_dir)
            )
            return data_ingestion_config

In [ ]:
import os
import zipfile
import gdown
from CNNClassifier.utils.common import get_size
from CNNClassifier.entity.config_entity import DataIngestionConfig

ModuleNotFoundError: No module named 'CNNClassifier'